<a href="https://colab.research.google.com/github/pedroafleite/landsat/blob/main/landsat_aws_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Landsat Satallite Images via AWS

## Finding satellite Images for your Area of Interest

**1. Enter search criteria**
- Go to [USGS explorer](https://earthexplorer.usgs.gov/) website
- Select your place of interest by providing Adress/Place

**2. Select your data set(s)**
- Toggle down 'Landsat 8'

**3. Additional criteria**
- Land Cloud Cover: Less than 10%
- Scene Cloud Cover: Less than 10%
- Collection Category: Tier 1
- Data Type Level-1: All
- Sensor Identifier: All

**4. Search Results**
- Copy the ID of the image you would like to use in your analysis
- Note down the path and row of the satellite imagery
- You might want to click the metadata button.




In [ ]:
#!pip install rasterio
#!pip install boto3

In [ ]:
import numpy as np
import boto3
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import imshow
import rasterio as rio
from rasterio.plot import show
from rasterio.session import AWSSession
import boto3

In [ ]:
! aws s3 ls s3://usgs-landsat/collection02/level-2/standard/oli-tirs/2020/026/027/LC08_L2SP_026027_20200827_20200906_02_T1/ --request-payer requester

/bin/bash: aws: command not found


The ID obtained was:

> LC08_L1TP_225067_20201203_20201218_02_T1

The URL path fis divided in the following structure: 

    Bucket/Collection/ProcessingLevel/Projection/Sensor/YearAcquired/Path/Row/Product.

Check metadata button.

First part is standard for all files:

    s3://usgs-landsat/

Second part stands for Collection (02), Level (01), Sensor (OLI_TIRS) and Year Acquired (2020), and only then Path (225) and Row (067).
    
    collection02/level-1/oli-tirs/2020/225/067/

Third part is the ID that was previously noted.

    LC08_L2SP_026027_20200827_20200906_02_T1

Last part is the repeated ID with two additional elements: (1) the Band you want to access (_B4 for instance) and (2) the image extension (.tiff).

    LC08_L2SP_026027_20200827_20200906_02_T1_SR_B2.TIF



## Load Cloud Optimized GeoTIFF (COG) Band File from the S3 Bucket

In [ ]:
cog = 's3://usgs-landsat/collection02/level-2/standard/oli-tirs/2020/026/027/LC08_L2SP_026027_20200827_20200906_02_T1/LC08_L2SP_026027_20200827_20200906_02_T1_SR_B2.TIF'

In [ ]:
with rio.Env(aws_session):
    with rio.open(cog) as src:
        profile = src.profile
        arr = src.read(1)

imshow(arr)

RasterioIOError: ignored

In [ ]:
# Check continuation at https://towardsdatascience.com/access-satellite-imagery-with-aws-and-google-colab-4660178444f5

src_image_array = arr.read(1)
src_image_array = src_image_array.astype(“f4”)
src_image_array

In [ ]:
src_image_array[src_image_array==0] = np.nan
fig, ax = plt.subplots(1, figsize=(12, 10))
show(src_image_array, ax=ax)
plt.show()

In [ ]:
# Window(col_off, row_off, width, height)
window = rio.windows.Window(1200, 1200, 750, 850)
subset = src_image.read(1, window=window)
fig, ax = plt.subplots(1, figsize=(12, 10))
show(subset, ax=ax)
ax.set_axis_off()
plt.show()

In [ ]:
rpath = ‘http://landsat-pds.s3.amazonaws.com/c1/L8/003/065/LC08_L1TP_003065_20190925_20191017_01_T1/LC08_L1TP_003065_20190925_20191017_01_T1_B4.TIF'
gpath = ‘http://landsat-pds.s3.amazonaws.com/c1/L8/003/065/LC08_L1TP_003065_20190925_20191017_01_T1/LC08_L1TP_003065_20190925_20191017_01_T1_B3.TIF'
bpath = ‘http://landsat-pds.s3.amazonaws.com/c1/L8/003/065/LC08_L1TP_003065_20190925_20191017_01_T1/LC08_L1TP_003065_20190925_20191017_01_T1_B2.TIF'
red = rio.open(rpath)
green = rio.open(gpath)
blue = rio.open(bpath)

In [ ]:
# Create an RGB image
with rio.open(‘RGB.tiff’,’w’,driver=’Gtiff’, width=red.width, height=red.height,count=3,crs=red.crs,transform=red.transform, dtype=red.dtypes[0]) as rgb:
rgb.write(blue.read(1),1)
rgb.write(green.read(1),2)
rgb.write(red.read(1),3)
rgb.close()

# Access Landsat with Google Earth Engine and Google Colab


In [ ]:
#!pip install google-api-python-client
#!python -c "from oauth2client import crypt"
#!pip install earthengine-api
!python -c "import ee; ee.Initialize()"
!earthengine authenticate

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ee/data.py", line 230, in get_persistent_credentials
    tokens = json.load(open(oauth.get_credentials_path()))
FileNotFoundError: [Errno 2] No such file or directory: '/root/.config/earthengine/credentials'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.6/dist-packages/ee/__init__.py", line 114, in Initialize
    credentials = data.get_persistent_credentials()
  File "/usr/local/lib/python3.6/dist-packages/ee/data.py", line 241, in get_persistent_credentials
    'Please authorize access to your Earth Engine account by '
ee.ee_exception.EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.
Instructions for updating:
non-resource variables are not supported in the long term
Running command usin

In [ ]:
import ee
ee.Initialize()
image = ee.Image('srtm90_v4')
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs': 'EPSG:4326', 'crs_transform': [0.000833333333333, 0, -180, 0, -0.000833333333333, 60]}], 'version': 1494271934303000.0, 'id': 'srtm90_v4', 'properties': {'system:time_start': 950227200000, 'system:time_end': 951177600000, 'system:asset_size': 18827626666}}


In [ ]:
# Get a download URL for an image.
image1 = ee.Image('srtm90_v4')
path = image1.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
})
path

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6372828bf91c925d155cb0ea8c792d61-c8ca0d55a8c5c3036cb3dfca90e23aea:getPixels'

GEE seems more straightforward than AWS.